In [1]:
%load_ext nb_mypy

Version 1.0.5


In [2]:
from loguru import logger
from s3path import S3Path
from datetime import datetime

from sm_pipelines_oo.config_loader.implementations import MockConfigLoader
from sm_pipelines_oo.steps.framework_processing_step import StepFactory
from sm_pipelines_oo.shared_config_schema import SharedConfig, Environment
from sm_pipelines_oo.aws_connector.concrete_connectors import create_aws_connector

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thomas-22/.config/sagemaker/config.yaml


In [3]:
_PROJECT_BUCKET = S3Path('/smp-oo-example')
_DATA_PREFIX = 'examples/data/'
output_filename = f'output-{datetime.now()}'

env: Environment = 'local'
_shared_config_dict={
    'project_name': 'test',
    'project_version': '0',
    'region': 'us-east-1',
    'project_bucket_name': _PROJECT_BUCKET.bucket,
    'role_name': 'sagemaker_pipelines_role'
}
_fw_processor_config_dict = {
    'step_name': 'preprocessing',
    'step_factory_class': 'FrameworkProcessor',
    'processor_init_config': {
        'framework_version': '0.23-1',
        'estimator_cls_name': 'SKLearn',
        'instance_count': 1,
        'instance_type': 'ml.m5.xlarge',
        'env': {'OUTPUT_FILENAME': output_filename},
    },
    'processor_run_config': {
        'code': 'preprocess.py',
        'source_dir': 'worker_code/preprocess',
        'inputs': {
            'input_1': (_PROJECT_BUCKET / _DATA_PREFIX / "input_1" ).as_uri(),
        },
        'outputs': {
            'output_1': (_PROJECT_BUCKET / _DATA_PREFIX / "output_1").as_uri(),
        },
    },
    'shared_config': _shared_config_dict
}
_shared_config = SharedConfig(**_shared_config_dict)
_aws_connector = create_aws_connector(
    shared_config=_shared_config,
    environment=env,
)
factory = StepFactory(
    step_config_dict=_fw_processor_config_dict,
    role_arn=_aws_connector.role_arn,
    sm_session=_aws_connector.sm_session,
    pipeline_session=_aws_connector.pipeline_session,
)

In [4]:
factory.run_processor()

INFO:sagemaker:Creating processing-job with name sklearn-2024-03-10-18-43-29-181
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting processing job
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-3b678:
    container_name: xpww6054qy-algo-1-3b678
    entrypoint:
    - /bin/bash
    - /opt/ml/processing/input/entrypoint/runproc.sh
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-3b678
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmp6nv4_4cg/algo-1-3b678/output:/opt/ml/output
    - /tmp/tmp6nv4_4cg/algo-1-3b678/config:/opt/ml/config
    - /tmp/tmp123gm30x:/opt/ml/processing/input_1
    - /tmp/tmp8rltxbjg:/opt/ml/processing/input/code/
    - /tm

time="2024-03-10T14:43:33-04:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmp6nv4_4cg\".\nSet `external: true` to use an existing network"
 Container xpww6054qy-algo-1-3b678  Creating
 Container xpww6054qy-algo-1-3b678  Created
Attaching to xpww6054qy-algo-1-3b678
xpww6054qy-algo-1-3b678  | Found existing installation: typing 3.7.4.3
xpww6054qy-algo-1-3b678  | Uninstalling typing-3.7.4.3:
xpww6054qy-algo-1-3b678  |   Successfully uninstalled typing-3.7.4.3
xpww6054qy-algo-1-3b678  | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
xpww6054qy-algo-1-3b678  | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/

INFO:sagemaker.local.image:===== Job Complete =====


In [5]:
from smp_oo_examples.validation import check_output

check_output(
    # s3_path=(_PROJECT_BUCKET / _DATA_PREFIX / "output_1" / output_filename).as_uri()
)

Success


In [6]:
print(datetime.now())

2024-03-10 14:43:40.271383
